In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('../../preproc/tmp/preproc.sqlite3')

In [22]:
c = conn.cursor()
c.execute("select * from task")

In [23]:
columns = [c for c, *_ in c.description]
columns

['id',
 'name',
 'status',
 'meta',
 'type',
 'error',
 'creation_date',
 'submission_date',
 'run_date',
 'done_date']

In [24]:
data = [row for row in c.fetchall()]

In [25]:
values = [dict(zip(columns, row)) for row in data]

In [27]:
import pandas as pd
import numpy as np
import json

def get_platform(meta):
    if 'platform' in meta:
        return meta['platform']
    elif 'task' in meta:
        return meta['task'].split('_')[1]
    else:
        return np.nan
    
data = (
    pd.DataFrame(values)
    .assign(meta=lambda d: d.meta.map(json.loads))
    .assign(samples=lambda d: d.meta.map(lambda m: m.get('samples', [])))
    .assign(platform=lambda d: d.meta.map(get_platform))
)
data

,creation_date,done_date,error,id,meta,name,run_date,status,submission_date,type,samples,platform
0,2015-08-21 18:46:27.696759,None,None,1,"{'samples': ['GSM1287918', 'GSM1287919', 'GSM1...",GSE53224_GPL570,None,done,None,preproc,"[GSM1287918, GSM1287919, GSM1287920, GSM128792...",GPL570
1,2015-08-21 18:46:27.698587,None,None,2,"{'samples': ['GSM1286097', 'GSM1286098', 'GSM1...",GSE53183_GPL570,None,done,None,preproc,"[GSM1286097, GSM1286098, GSM1286099, GSM128610...",GPL570
2,2015-08-21 18:46:27.699732,None,None,3,"{'samples': ['GSM1283115', 'GSM1283116', 'GSM1...",GSE53157_GPL570,None,done,None,preproc,"[GSM1283115, GSM1283116, GSM1283117, GSM128311...",GPL570
3,2015-08-21 18:46:27.700555,None,None,4,"{'samples': ['GSM1603234', 'GSM1603235', 'GSM1...",GSE65721_GPL5082,None,skip,None,preproc,"[GSM1603234, GSM1603235, GSM1603236, GSM160323...",GPL5082
4,2015-08-21 18:46:27.701362,None,None,5,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GSE65721_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
5,2015-08-21 18:46:27.702123,None,None,6,"{'samples': ['GSM1282316', 'GSM1282317', 'GSM1...",GSE53092_GPL570,None,done,None,preproc,"[GSM1282316, GSM1282317, GSM1282318, GSM1282319]",GPL570
6,2015-08-21 18:46:27.702899,None,None,7,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GSE65707_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
7,2015-08-21 18:46:27.703622,None,None,8,"{'samples': ['GSM1281432', 'GSM1281433', 'GSM1...",GSE53059_GPL570,None,done,None,preproc,"[GSM1281432, GSM1281433, GSM1281434, GSM128143...",GPL570
8,2015-08-21 18:46:27.704425,None,None,9,"{'samples': ['GSM1281025', 'GSM1281026', 'GSM1...",GSE53046_GPL570,None,done,None,preproc,"[GSM1281025, GSM1281026, GSM1281027, GSM128102...",GPL570
9,2015-08-21 18:46:27.705136,None,None,10,"{'samples': ['GSM1280329', 'GSM1280330', 'GSM1...",GSE53012_GPL570,None,done,None,preproc,"[GSM1280329, GSM1280330, GSM1280331, GSM128033...",GPL570


In [31]:
data.query("type == 'convert-merge'").platform.value_counts()

GPL570    3672
GPL96      693
GPL571     436
dtype: int64

In [30]:
data.query("type == 'preproc' and status == 'done'").platform.value_counts()[:10]

GPL570     3692
GPL1261    3113
GPL96       693
GPL571      436
GPL8321     395
GPL4133     248
GPL6480     148
GPL90       105
GPL4091      59
GPL3921      56
dtype: int64

In [32]:
converted_tasks = (
    pd.DataFrame(data
     .query("type == 'convert-merge' and status=='done' ")
     .apply(lambda x: x['meta']['task'], axis=1),
    columns=['task']             
    )
    .assign(platform=lambda d: d['task'].map(lambda t: t.split('_')[1]))
    .assign(series=lambda d: d['task'].map(lambda t: t.split('_')[0]))
    .query("platform != 'GPL570'")
)
converted_tasks[['task']].to_csv('/Users/nikita/mnt/hpc.mipt.ru/converted-96-571.csv', header=None, index=False)
converted_tasks.head()

,task,platform,series
17514,GSE10040_GPL571,GPL571,GSE10040
17515,GSE10086_GPL571,GPL571,GSE10086
17516,GSE10087_GPL571,GPL571,GSE10087
17517,GSE10088_GPL571,GPL571,GSE10088
17518,GSE10096_GPL571,GPL571,GSE10096


In [ ]:
converted_tasks.platform.value_counts()

In [ ]:
converted_tasks[['series', 'platform']].to_csv('../data/preproc/converted_tasks.csv', index=False)

# GNUSE

In [35]:
to_edit = pd.read_csv('/Users/nikita/mnt/hpc.mipt.ru/bad_samples.csv')
to_edit.head()

,sample,series,platform
0,GSM252822.gnuse,GSE10006,GPL570
1,GSM252824.gnuse,GSE10006,GPL570
2,GSM252826.gnuse,GSE10006,GPL570
3,GSM253283.gnuse,GSE10024,GPL96
4,GSM253664.gnuse,GSE10041,GPL570


In [68]:
to_edit.platform.value_counts()

GPL570    13132
GPL96      2391
GPL571     1313
dtype: int64

In [44]:
to_edit_series= (
    pd.DataFrame(to_edit
                 .groupby(['series', 'platform'], as_index=False)
                 .sample
                 .apply(list), columns=['samples'])
    .reset_index()
    .assign(name=lambda d: d['series'].combine(d['platform'], func='{}_{}_convert.txt'.format))
    .set_index('name')
)
to_edit_series

,series,platform,samples
name,,,
GSE10006_GPL570_convert.txt,GSE10006,GPL570,"[GSM252822.gnuse, GSM252824.gnuse, GSM252826.g..."
GSE10024_GPL96_convert.txt,GSE10024,GPL96,[GSM253283.gnuse]
GSE10041_GPL570_convert.txt,GSE10041,GPL570,"[GSM253664.gnuse, GSM253666.gnuse, GSM253667.g..."
GSE10072_GPL96_convert.txt,GSE10072,GPL96,"[GSM254639.gnuse, GSM254677.gnuse, GSM254709.g..."
GSE10097_GPL96_convert.txt,GSE10097,GPL96,[GSM255006.gnuse]
GSE10135_GPL570_convert.txt,GSE10135,GPL570,"[GSM252822.gnuse, GSM252824.gnuse, GSM252826.g..."
GSE10137_GPL96_convert.txt,GSE10137,GPL96,[GSM256259.gnuse]
GSE10147_GPL570_convert.txt,GSE10147,GPL570,"[GSM256745.gnuse, GSM256746.gnuse, GSM256747.g..."
GSE10161_GPL96_convert.txt,GSE10161,GPL96,[GSM256951.gnuse]


In [45]:
to_edit_series.loc["GSE8730_GPL570_convert.txt"]

series                GSE8730
platform               GPL570
samples     [GSM216539.gnuse]
Name: GSE8730_GPL570_convert.txt, dtype: object

In [66]:
to_edit_series.to_pickle('/Users/nikita/mnt/hpc.mipt.ru/to_edit_series.py.pickle')

In [69]:
to_edit_series.shape

(1337, 3)

In [65]:
root = '/Users/nikita/Sources/python-notebooks/data/preproc/'
copy_root = '/tmp/'

from os.path import join, exists
from os import makedirs
import shutil
import csv

for name, samples in  to_edit_series.samples.iteritems():
    if name != "GSE8730_GPL570_convert.txt":
        continue
    file_name = join(root, name.replace('.txt', ''), 'out', name)    
    if not exists(file_name):
        print('{} not exists. Skipping'.format(file_name))
        continue
        
    shutil.copyfile(file_name, join(copy_root, name))    
    d = pd.read_table(file_name, sep=' ', index_col=0)
    for sample in samples:
        sample_name = sample.replace('.gnuse', '.expr')
        if sample_name in d.columns:
            del d[sample_name]
    d.to_csv(file_name, sep=' ', quoting=csv.QUOTE_NONNUMERIC, index_label=False)
    

# Agilent

In [70]:
agilent_platforms = ['GPL4133', 'GPL6480', 'GPL15849', 'GPL1708', 'GPL887', 'GPL4091', 'GPL9128', 'GPL7264', 'GPL11387',
             'GPL8687', 'GPL6848', 'GPL2879', 'GPL5477', 'GPL8841', 'GPL10123', 'GPL4093', 'GPL11386', 'GPL10806',
             'GPL8269', 'GPL10150', 'GPL8583', 'GPL15931', 'GPL4126', 'GPL10152', 'GPL16050', 'GPL2567', 'GPL9053',
             'GPL14550', 'GPL5325', 'GPL10808', 'GPL13691', 'GPL9075', 'GPL8736', 'GPL885', 'GPL9777', 'GPL7504',
             'GPL8693', 'GPL2873', 'GPL17077', 'GPL10734', 'GPL13953', 'GPL13607', 'GPL13685', 'GPL7015', 'GPL15560',
             'GPL18623', 'GPL10481', 'GPL16280', 'GPL8737', 'GPL11068']


In [71]:
data

,creation_date,done_date,error,id,meta,name,run_date,status,submission_date,type,samples,platform
0,2015-08-21 18:46:27.696759,None,None,1,"{'samples': ['GSM1287918', 'GSM1287919', 'GSM1...",GSE53224_GPL570,None,done,None,preproc,"[GSM1287918, GSM1287919, GSM1287920, GSM128792...",GPL570
1,2015-08-21 18:46:27.698587,None,None,2,"{'samples': ['GSM1286097', 'GSM1286098', 'GSM1...",GSE53183_GPL570,None,done,None,preproc,"[GSM1286097, GSM1286098, GSM1286099, GSM128610...",GPL570
2,2015-08-21 18:46:27.699732,None,None,3,"{'samples': ['GSM1283115', 'GSM1283116', 'GSM1...",GSE53157_GPL570,None,done,None,preproc,"[GSM1283115, GSM1283116, GSM1283117, GSM128311...",GPL570
3,2015-08-21 18:46:27.700555,None,None,4,"{'samples': ['GSM1603234', 'GSM1603235', 'GSM1...",GSE65721_GPL5082,None,skip,None,preproc,"[GSM1603234, GSM1603235, GSM1603236, GSM160323...",GPL5082
4,2015-08-21 18:46:27.701362,None,None,5,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GSE65721_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
5,2015-08-21 18:46:27.702123,None,None,6,"{'samples': ['GSM1282316', 'GSM1282317', 'GSM1...",GSE53092_GPL570,None,done,None,preproc,"[GSM1282316, GSM1282317, GSM1282318, GSM1282319]",GPL570
6,2015-08-21 18:46:27.702899,None,None,7,"{'samples': ['GSM1603354', 'GSM1603355', 'GSM1...",GSE65707_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
7,2015-08-21 18:46:27.703622,None,None,8,"{'samples': ['GSM1281432', 'GSM1281433', 'GSM1...",GSE53059_GPL570,None,done,None,preproc,"[GSM1281432, GSM1281433, GSM1281434, GSM128143...",GPL570
8,2015-08-21 18:46:27.704425,None,None,9,"{'samples': ['GSM1281025', 'GSM1281026', 'GSM1...",GSE53046_GPL570,None,done,None,preproc,"[GSM1281025, GSM1281026, GSM1281027, GSM128102...",GPL570
9,2015-08-21 18:46:27.705136,None,None,10,"{'samples': ['GSM1280329', 'GSM1280330', 'GSM1...",GSE53012_GPL570,None,done,None,preproc,"[GSM1280329, GSM1280330, GSM1280331, GSM128033...",GPL570
